In [17]:
# LOADING DATA AND PREPROCESSING

In [18]:
# STEP 1 - IMPORTING THE LIBRARIES

# For matrix calculations and Data Management
import numpy as np

#Importing libraries required for the model
import tensorflow as tf
import keras
import keras.backend as K

from tensorflow.keras.optimizers import SGD, Adam, Adagrad, RMSprop
from keras.applications import *
from keras.preprocessing import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, BatchNormalization, Dropout
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

#for plotting charts used for data visualizations
import matplotlib.pyplot as plt

#Libtraries for Locating and loading data
import glob
from PIL import Image 
import os
from os import listdir

In [21]:
# STEP 2 - MAKING A LIST OF PATHS TO ALL FOLDERS WHERE WE HAVE OUR DATA

# setting path to our dataset folder
dirName=("augmented_data")
folders=listdir(dirName)
#Getting the names for all the folders containing the data
def getListOfFiles(dirName):
    #create a list of sub directories and files (if any)
    #names in the given directory
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for fol_name in listofFile:
        fullPath=os.path.join(dirNmae, fol_name)
        allFiles.append(fullPath)
        return allFiles
    Folders=getListOfFiles(dirName)
    len(Folders)
    subfolders=[]
    for num in range(len(Folders)):
        sub_fols=getListOfFiles(Folders[num])
        subfolders+=sub_fols
        #Now the subfolders contains the address to all our data folders for each class
        subfolders
# STEP 2(i)- LOADING THE IMAGES INTO THE MACHINE UNDERSTANDABLE DATA
        # Loading the data and preprocessing it to make it in trainable format
        # X data includes the data generated for each image 
        # Y data includes ID No, unique for every different species, so we have 6 classes
        # which will the label
        X_data=[]
        Y_data=[]
        id_no=0
        # to make a list of tuples, where we'll store the info about the image, category and species
        found=[]
        # iterating in all folders under Augmneted data folder
        for paths in subfolders:
            # set folder path for each unique class and category
            files=glob.glob(paths+"/*.jpg")
            # adding tuples to the list that contains folder name and subfolder name
            found.append((paths.split('\\')[-2],paths.spilt('\\')[-1]))
            # iterating all the files under the folder one by one
            for myFile in files:
                img=Image.open(myFile)
                img=img.resize((224,224), Image.ANTIALIAS) #resizes image without ratio
                # converts the image to numpy arrays
                img=np.array(img)
                if img.shape==(224,224,3): 
                    # add the numpy images to the matrix with all the data
                    X_data.append(img)
                    Y_data.append(id_no)
                    id_no+=1
# STEP 2(ii) - DATA SPLITTING INTO THE TRAIN AND TEST to see our data 
                    print(X_data)
                    print(Y_data)
                    # Converting lists to np arrays again
                    X=np.arrays(X_data)
                    Y=np.arrays(Y_data)
                    # Print shapes to see if they are correct
                    print("x-shape",X.shape,"y-shape", Y.shape)
                    X=X.astype('float32')/255.0
                    # keras library offers a function called to_categorical() or OneHotEncoder
                    #integer data. The sequence has an example of all known values so we use the to_categorical() function directly
                    y_cat=to_categorical(Y_data,len(subfolders))
                    print("X shape",X,"y_cat shape",y_cat)
                    print("X shape",X.shape,"y_cat shape", y_cat.shape)
                    X_train, X_test, y_train, y_test=tain_test_split(X,y_cat,test_size=0.2)
                    print("The model has"+str(len(X_train))+"inputs")
                    
# STEP 3 - BUILDING A CONVOLUTION NEURAL NETWORKS
                    
                    # STEP 3(i) - Sequential
                    
                    early_stop_loss=EarlyStopping(monitor='loss',patience=3,verbose=1)
                    early_stop_val_acc=EarlyStopping(monitor='val_accuracy',patience=3,verbose=1)
                    model_callbacks=[early_stop_loss,early_stop_val_acc]
                    
                    # STEP 3(ii) ADDING LAYERS (Conv, Maxpool, Flatten, Dense, Dropout)
                    
                    # defining our model, All the layers and configurations
                    
                    def load_CNN(output_size):
                        K.clear_session()
                        model=Sequential()
                        model.add(Dropout(0.4,input_shape=(224,224,3)))
                        model.add(Conv2D(256,(5,5),input_shape=(224, 224, 3), activation='relu'))
                        model.add(MaxPool2D(pool_size=(2,2)))
                        
                        #model.add(BatchNormalization())
                        model.add(Conv2D(128,(3,3), activation='relu'))
                        model.add(MaxPool2D(pool_size=(2,2)))
                        #model.add(BatchNormalization())
                        model.add(Conv2D(64,(3,3), activation='relu'))
                        model.add(MaxPool2D(pool_size=(2,2)))
                        #model.add(BatchNormalization())
                        model.add(Flatten())
                        model.add(Dense(512, activation='relu'))
                        model.add(Dropout(0.3))
                        model.add(Dense(256, activation='relu'))
                        model.add(Dropout(0.3))
                        model.add(Dense(128, activation='relu'))
                        model.add(Dropout(0.3))
                        model.add(Dense(output_size, activation='softmax'))
                        return model
                    
# STEP 4- BUILDING MODEL SUMMARY

                    # STEP 4(i) - Building a model based on the above defined function
                    
                    model=load_CNN(6) # Number of Columns/ Outputs
                    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
                    model.summary() # to print the model summary
                    weights=model.get_weights() # to get the weights from the created model
                    
                    # STEP 4(ii) - FITTING THE MODEL
                    # some arrays to store the result of each model (model trained on each bath size)
                    
                    histories_acc=[]
                    histories_val_acc=[]
                    histories_loss=[]
                    histories_val_loss=[]
                    model.set_weights(weights)
                    h=model.fit(X_train,y_train,
                                batch_size=16,
                                epochs=7,
                                verbose=1,
                                callbacks=[early_stop_loss],
                                shuffle=True,
                                validation_dat=(X_test, y_test))
                    model.summary()
# STEP 5 - EVALUATION: (i) ACCURACY
                   
                    # Printing the keys we have for the stores values
                    print(h.history.keys())
                    
                    # Appendind the data for each epochh in an arr and for each batch size
                    
                    histories_acc.append(h.history['acc'])
                    histories_val_acc.append(h.history['val_acc'])
                    histories_loss.append(h.history['loss'])
                    histories_val_loss.append(h.history['val_loss'])
                    
                    # Converting into numpy arrays
                    
                    histories_acc=np.array(histories_acc)
                    histories_val_acc=np.array(histories_val_acc)
                    histories_loss=np.array(histories_loss)
                    histories_val_loss=np.array(histories_val_loss)
                    
                    # Here we have 3 columns and 6 rows each, ever row represents different batch sizes and every coloumn represents different epoch scores
                    print('histories_acc',histories_acc,
                           'histories_loss',histories_loss,
                           'histories_val_acc',histories_val_acc,
                           'histories_val_loss',histories_val_loss)
                    
# PREDICTION
# STEP 5(ii) - LOADING A TEST IMAGE AND MAKING A TEST PREDICTION
                    
    # Predicting the image's classes
    # Individual scores for each calss as well as class with the highest score is printed 
    # Making predictions, storing teh result as an array of probabilities of each class predicted
                    
                    predictions=model.predict_proba([X_test[image_number].reshape(1,224,224,3)])
                    for idx, result, x in zip(range(0,6),found, predictions[0]):
                        print("Label: {}, Type : {}, Species : {}, Score : {}%".format(idx,result[0],result[1], round(x*100,3)))
                        #predictiong the class with max probabilty
                        ClassIndex=model.predict_classes([X_test[image_number].reshape(1,224,224,3)])
                        #getting the index of th eclass which we can pass to the boat_types list to get the boat type name
                        ClassIndex
                        #printing the final output
                        print(found[ClassIndex[0]])
                        # Loading the test data
                        image_number =np.random.randint(0,len(X_test))
                        print(image_number)
                        # Plotting the test image
                        plt.figure(figsize=(8,8))
                        plt.imshow(X_test[image_number])

In [22]:
# STEP 6 - SAVE THE MODEL
                        
model_json=model.to_json() #indent=2
with open("final_model.json","w") as json_file:
    json_file.write(model_json)
    
    # serializing the weights to H5
    
    model.save_weights("final_model.h5")
    print("Saved model to disk")

Saved model to disk
